In [11]:
from tensorflow.keras.preprocessing import text
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.utils import pad_sequences
from tensorflow.keras.utils import to_categorical
import numpy as np
import pandas as pd

In [12]:
#taking random sentences as data
data = """Deep learning (also known as deep structured learning) is part of a broader family of machine learning methods based on artificial neural networks with representation learning. Learning can be supervised, semi-supervised or unsupervised.
Deep-learning architectures such as deep neural networks, deep belief networks, deep reinforcement learning, recurrent neural networks, convolutional neural networks and Transformers have been applied to fields including computer vision, speech recognition, natural language processing, machine translation, bioinformatics, drug design, medical image analysis, climate science, material inspection and board game programs, where they have produced results comparable to and in some cases surpassing human expert performance.
"""
dl_data = data.split()

In [13]:
#tokenization
tokenizer = text.Tokenizer()
tokenizer.fit_on_texts(dl_data)
word2id = tokenizer.word_index

word2id['PAD'] = 0
id2word = {v:k for k, v in word2id.items()}
wids = [[word2id[w] for w in text.text_to_word_sequence(doc)] for doc in dl_data]

vocab_size = len(word2id)
embed_size = 100
window_size = 2

print('Vocabulary Size:', vocab_size)
print('Vocabulary Sample:', list(word2id.items())[:10])

Vocabulary Size: 75
Vocabulary Sample: [('learning', 1), ('deep', 2), ('networks', 3), ('neural', 4), ('and', 5), ('as', 6), ('of', 7), ('machine', 8), ('supervised', 9), ('have', 10)]


In [14]:
#generating (context word, target/label word) pairs
def generate_context_word_pairs(corpus, window_size, vocab_size):
    context_length = window_size*2
    for words in corpus:
        sentence_length = len(words)
        for index, word in enumerate(words):
            context_words = []
            label_word   = []
            start = index - window_size
            end = index + window_size + 1

            context_words.append([words[i]
                                 for i in range(start, end)
                                 if 0 <= i < sentence_length
                                 and i != index])
            label_word.append(word)

            x = pad_sequences(context_words, maxlen=context_length)
            y = to_categorical(label_word, vocab_size)
            yield (x, y)

i = 0
for x, y in generate_context_word_pairs(corpus=wids, window_size=window_size, vocab_size=vocab_size):
    if 0 not in x[0]:
        # print('Context (X):', [id2word[w] for w in x[0]], '-> Target (Y):', id2word[np.argwhere(y[0])[0][0]])

        if i == 10:
            break
        i += 1

In [15]:
#model building
import tensorflow.keras.backend as K
from keras.models import Sequential
from keras.layers import Dense, Embedding, Lambda

cbow = Sequential()
cbow.add(Embedding(input_dim=vocab_size, output_dim=embed_size, input_length=window_size*2))
cbow.add(Lambda(lambda x: K.mean(x, axis=1), output_shape=(embed_size,)))
cbow.add(Dense(vocab_size, activation='softmax'))
cbow.compile(loss='categorical_crossentropy', optimizer='rmsprop')

print(cbow.summary())

# from IPython.display import SVG
# from keras.utils.vis_utils import model_to_dot

# SVG(model_to_dot(cbow, show_shapes=True, show_layer_names=False, rankdir='TB').create(prog='dot', format='svg'))

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lambda_2 (Lambda)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [16]:
for epoch in range(1, 6):
    loss = 0.
    i = 0
    for x, y in generate_context_word_pairs(corpus=wids, window_size=window_size, vocab_size=vocab_size):
        i += 1
        loss += cbow.train_on_batch(x, y)
        if i % 100000 == 0:
            print('Processed {} (context, word) pairs'.format(i))

    print('Epoch:', epoch, '\tLoss:', loss)
    print()

Epoch: 1 	Loss: 431.32727

Epoch: 2 	Loss: 430.7813

Epoch: 3 	Loss: 429.0235

Epoch: 4 	Loss: 427.27524

Epoch: 5 	Loss: 425.75864



In [17]:
weights = cbow.get_weights()[0]
weights = weights[1:]
print(weights.shape)

pd.DataFrame(weights, index=list(id2word.values())[1:]).head()

(74, 100)


,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
deep,-0.001380,-0.052387,-0.020369,-0.003171,0.006323,-0.009616,0.023566,-0.011275,-0.021994,-0.032643,...,0.001856,-0.005692,0.031364,-0.064365,0.028500,-0.002178,0.049791,-0.057154,-0.004367,-0.017154
networks,-0.051292,0.000265,0.023165,0.000180,0.030213,-0.014857,0.010475,0.032862,-0.019126,0.033064,...,-0.037537,-0.013482,-0.065375,0.010949,0.031635,0.027153,-0.014851,-0.060854,0.016581,-0.048326
neural,-0.006556,-0.006218,-0.013444,-0.015961,0.016594,0.021871,0.012963,-0.034647,-0.026203,0.010552,...,-0.003700,0.043730,-0.005246,-0.014007,0.012131,-0.035186,0.029934,-0.009042,-0.042141,0.008936
and,-0.043231,0.035912,-0.008876,0.030779,-0.040348,0.014713,0.027900,-0.043834,-0.011921,-0.035915,...,-0.035442,0.023745,-0.001483,0.038473,-0.021467,0.019912,0.000988,0.010163,0.044667,0.001026
as,-0.036716,-0.014763,0.040019,0.040291,0.006574,0.045934,-0.045928,0.035303,0.014310,0.012535,...,-0.028998,-0.001662,0.034958,-0.044080,0.005235,-0.017087,-0.011756,0.013145,-0.018816,-0.035061


In [ ]:
'''


## 🔶 PRACTICAL NAME:

**“Implement the Continuous Bag of Words (CBOW) for text recognition in a given dataset.”**

---

## 🔹 OVERVIEW / THEORY OF CBOW

### What is CBOW?

CBOW (Continuous Bag of Words) is a **word embedding technique** proposed in the **Word2Vec** model by **Mikolov et al. (2013)**.

It’s a **neural network model** that learns to **predict a target word given its surrounding context words**.

---

### 🔸 Core Concept:

* **Input:** Context words (surrounding words)
* **Output:** Target word (the word that fits best in that context)

Example:
Sentence → “The cat sits on the mat”
If window size = 2, and target = “sits”,
context words = [“The”, “cat”, “on”, “the”]

CBOW tries to **predict "sits"** from these context words.

---

### 🔸 Working Mechanism (Step-by-Step):

1. **Text Preprocessing:**

   * Tokenize text (convert words → integers)
   * Create vocabulary

2. **Context & Target Generation:**

   * For each word, select its neighboring words (context)
   * Example (window = 2):
     Text: “The quick brown fox jumps”
     Pairs:
     (“The”, “brown”) → “quick”
     (“quick”, “fox”) → “brown”
     etc.

3. **Model Architecture:**

   * **Input layer:** Context words (one-hot encoded)
   * **Hidden layer:** Dense layer with embedding dimension
   * **Output layer:** Softmax layer to predict target word

4. **Training Objective:**

   * The model learns **word vectors (embeddings)** that make semantically similar words closer in vector space.
   * **Loss:** Cross-entropy loss
   * **Optimizer:** SGD / Adam

---

### 🔸 Applications of CBOW:

* Word embeddings for NLP
* Text classification
* Sentiment analysis
* Machine translation
* Speech/text recognition

---

### 🔸 CBOW vs Skip-Gram:

| Feature   | CBOW                     | Skip-Gram           |
| --------- | ------------------------ | ------------------- |
| Predicts  | Target word from context | Context from target |
| Training  | Faster                   | Slower              |
| Good for  | Frequent words           | Rare words          |
| Direction | Context → Target         | Target → Context    |

---

## 🔶 CODE EXPLANATION (CELL BY CELL)

### 🧩 **Cell 1: Import Libraries**

```python
from tensorflow.keras.preprocessing import text
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.utils import pad_sequences
from tensorflow.keras.utils import to_categorical
import numpy as np
import pandas as pd
```

**Purpose:**
Imports tools for text preprocessing, sequence padding, and categorical encoding.

**Explanation:**

* `text` – for tokenizing sentences
* `sequence` & `pad_sequences` – make all input sequences of equal length
* `to_categorical` – converts word indices into one-hot encoded form
* `numpy`, `pandas` – used for handling data arrays

**Possible Viva Questions:**

* Why use `pad_sequences`?
* What is one-hot encoding?
* Why is text tokenization important?

---

### 🧩 **Cell 2: Dataset Creation**

```python
#taking random sentences as data
data = """Deep learning (also known as deep structured learning) is part of a broader family...
"""
dl_data = data.split()
```

**Purpose:**
Creates a simple custom dataset (manually written text).

**Explanation:**

* Multiline string → acts as corpus
* `split()` → breaks into individual words

**Viva Questions:**

* Why did you create a custom dataset?
* How does `.split()` tokenize the data?
* What could be an alternative tokenizer?

---

### 🧩 **Cell 3: Tokenization and Vocabulary Creation**

```python
tokenizer = text.Tokenizer()
tokenizer.fit_on_texts(dl_data)
word2idx = tokenizer.word_index
V = len(word2idx)
```

**Purpose:**
Converts text into numerical format.

**Explanation:**

* `Tokenizer()` → assigns each unique word an integer
* `fit_on_texts()` → builds vocabulary
* `word_index` → dictionary {word: index}
* `V` → vocabulary size

**Viva Questions:**

* What is vocabulary size?
* Why do we convert words into numbers?
* What is the role of `Tokenizer`?

---

### 🧩 **Cell 4: Creating Context-Target Pairs**

```python
window_size = 2
dataX = []
dataY = []

for i in range(window_size, len(dl_data) - window_size):
    context = []
    for j in range(-window_size, window_size + 1):
        if j != 0:
            context.append(tokenizer.texts_to_sequences([dl_data[i + j]])[0][0])
    dataX.append(context)
    dataY.append(tokenizer.texts_to_sequences([dl_data[i]])[0][0])
```

**Purpose:**
Generates input (context) and output (target) pairs for CBOW training.

**Explanation:**

* Loop over words in corpus
* Select 2 words before and after as context
* Skip the target word itself
* Convert each word into its integer ID

**Viva Questions:**

* What is a context window?
* How do you choose window size?
* Why exclude the center word?

---

### 🧩 **Cell 5: Convert Data to NumPy Arrays**

```python
dataX = np.array(dataX)
dataY = np.array(dataY)
```

Simple conversion for training compatibility.

**Viva Question:**

* Why use NumPy arrays instead of lists?

---

### 🧩 **Cell 6: Model Building**

```python
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Lambda, Dense
import tensorflow.keras.backend as K

model = Sequential()
model.add(Embedding(V+1, 10, input_length=window_size*2))
model.add(Lambda(lambda x: K.mean(x, axis=1), output_shape=(10,)))
model.add(Dense(V, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
```

**Purpose:**
Implements the CBOW neural architecture.

**Explanation:**

* `Embedding(V+1, 10)` → word embedding layer (10-dimensional vectors)
* `Lambda(...mean...)` → averages context embeddings
* `Dense(V, softmax)` → predicts target word probability
* `compile()` → defines optimizer & loss function

**Viva Questions:**

* What does the Embedding layer do?
* Why use `Lambda` with `K.mean()`?
* Why choose ‘softmax’ at the output?
* What loss function is used in CBOW?

---

### 🧩 **Cell 7: Prepare Outputs for Training**

```python
y = to_categorical(dataY - 1, V)
```

**Purpose:**
Converts target word indices into one-hot encoded vectors.

**Explanation:**

* `dataY - 1` because tokenizer indices start from 1
* `to_categorical()` → convert to one-hot encoding

**Viva Question:**

* Why subtract 1 from `dataY`?
* Why use one-hot encoding for output?

---

### 🧩 **Cell 8: Train the Model**

```python
model.fit(dataX, y, epochs=500, verbose=0)
```

**Purpose:**
Train CBOW model using context-target pairs.

**Explanation:**

* `epochs=500` – trains the model 500 times
* `verbose=0` – hides training progress
* Model learns word relationships

**Viva Questions:**

* What does one epoch mean?
* Why choose 500 epochs?
* What happens during model training?

---

## 🔶 POSSIBLE ADVANCED QUESTIONS (by Chief Examiner)

| Question                                               | Answer                                                                             |
| ------------------------------------------------------ | ---------------------------------------------------------------------------------- |
| What is the main objective of CBOW?                    | To predict a target word given surrounding context words.                          |
| Why is it called “Bag of Words”?                       | Because order of context words doesn’t matter — only their presence.               |
| How are embeddings learned?                            | By minimizing loss between predicted and actual target word (via backpropagation). |
| How can you visualize embeddings?                      | Using dimensionality reduction methods like PCA or t-SNE.                          |
| What is the difference between one-hot and embeddings? | One-hot is sparse (high-dimensional), embeddings are dense (low-dimensional).      |
| Can CBOW handle OOV (Out-of-Vocabulary) words?         | No, only words seen during training have embeddings.                               |
| Why use softmax activation?                            | It converts output scores into probabilities for each word.                        |
| What happens if window size increases too much?        | It may include irrelevant context and increase noise.                              |

---

## 🔶 SUMMARY (For Quick Revision)

* **CBOW** = Predict center word from context
* **Architecture:** Embedding → Mean → Softmax
* **Loss:** Categorical Cross-Entropy
* **Optimizer:** Adam
* **Core goal:** Learn dense word embeddings
* **Alternative:** Skip-Gram (reverse task)
* **Applications:** Text recognition, NLP, translation





Perfect ✅ — since your practical exam is **“Implement the Continuous Bag of Words (CBOW) for text recognition”**, I’ll now give a **line-by-line explanation** of *every code cell* in your notebook.

This will be in **simple viva-ready format** — each code line explained clearly (what it does, why it’s used, and how it fits in the CBOW model).

---

## 🧩 **CELL 1 — Importing Required Libraries**

```python
from tensorflow.keras.preprocessing import text
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.utils import pad_sequences
from tensorflow.keras.utils import to_categorical
import numpy as np
import pandas as pd
```

### 🔹 Line-by-Line Explanation:

1. `from tensorflow.keras.preprocessing import text`
   → Imports text preprocessing utilities (mainly for tokenization and text-to-sequence conversion).

2. `from tensorflow.keras.preprocessing import sequence`
   → Used to manage sequences (padding/truncating sequences of words).

3. `from tensorflow.keras.utils import pad_sequences`
   → Specifically used to make all input sequences of the same length (by adding zeros if needed).

4. `from tensorflow.keras.utils import to_categorical`
   → Converts numerical labels into one-hot encoded form (used for categorical prediction).

5. `import numpy as np`
   → NumPy provides support for arrays and numerical operations.

6. `import pandas as pd`
   → Pandas is imported for data manipulation (though not heavily used here).

---

## 🧩 **CELL 2 — Preparing the Text Dataset**

```python
#taking random sentences as data
data = """Deep learning (also known as deep structured learning) is part of a broader family of machine learning methods based on artificial neural networks with representation learning. Learning can be supervised, semi-supervised or unsupervised.
Deep-learning architectures such as deep neural networks, deep belief networks, deep reinforcement learning, recurrent neural networks, convolutional neural networks and Transformers have been applied to fields including computer vision, speech recognition, natural language processing, machine translation, bioinformatics, drug design, medical image analysis, climate science, material inspection and board game programs, where they have produced results comparable to and in some cases surpassing human expert performance.
"""
dl_data = data.split()
```

### 🔹 Line-by-Line Explanation:

1. `#taking random sentences as data`
   → This is a comment explaining that a random paragraph is used as a dataset.

2. `data = """Deep learning ..."""`
   → Defines a multiline string containing a paragraph of text about deep learning.

3. `dl_data = data.split()`
   → Splits the paragraph into individual words based on spaces.
   → The result is a list of words, e.g. `["Deep", "learning", "also", "known", ...]`.

**Purpose:** This creates a simple corpus (collection of words) for training the CBOW model.

---

## 🧩 **CELL 3 — Tokenization and Vocabulary Creation**

```python
tokenizer = text.Tokenizer()
tokenizer.fit_on_texts(dl_data)
word2idx = tokenizer.word_index
V = len(word2idx)
```

### 🔹 Line-by-Line Explanation:

1. `tokenizer = text.Tokenizer()`
   → Creates a tokenizer object that will map each unique word to an integer.

2. `tokenizer.fit_on_texts(dl_data)`
   → The tokenizer reads all the words and builds an internal vocabulary.
   → Each unique word is assigned a unique integer ID.

3. `word2idx = tokenizer.word_index`
   → Creates a Python dictionary where keys = words and values = their integer indices.
   Example: `{‘deep’: 1, ‘learning’: 2, ...}`

4. `V = len(word2idx)`
   → Calculates vocabulary size (`V`) = number of unique words.

**Purpose:** Converts text words → numerical IDs for neural network processing.

---

## 🧩 **CELL 4 — Creating Context and Target Word Pairs**

```python
window_size = 2
dataX = []
dataY = []

for i in range(window_size, len(dl_data) - window_size):
    context = []
    for j in range(-window_size, window_size + 1):
        if j != 0:
            context.append(tokenizer.texts_to_sequences([dl_data[i + j]])[0][0])
    dataX.append(context)
    dataY.append(tokenizer.texts_to_sequences([dl_data[i]])[0][0])
```

### 🔹 Line-by-Line Explanation:

1. `window_size = 2`
   → Defines how many words before and after the target word are used as context.
   Example: if window=2 and target="learning", context = [‘Deep’, ‘(also’, ‘as’, ‘deep’]

2. `dataX = []` and `dataY = []`
   → Initialize empty lists to store context (`dataX`) and target (`dataY`) pairs.

3. `for i in range(window_size, len(dl_data) - window_size):`
   → Loops through each word, leaving space at start & end for context words.

4. `context = []`
   → Temporary list to store context words for the current target word.

5. `for j in range(-window_size, window_size + 1):`
   → Loops from `-2` to `+2` (for window size 2).

6. `if j != 0:`
   → Skips the center word (because that’s the target).

7. `context.append(tokenizer.texts_to_sequences([dl_data[i + j]])[0][0])`
   → Converts each context word into its numeric ID using tokenizer and appends to `context` list.

8. `dataX.append(context)`
   → Adds the context (list of 4 integers) to the training input data list.

9. `dataY.append(tokenizer.texts_to_sequences([dl_data[i]])[0][0])`
   → Adds the target word’s numeric ID to the output list.

**Purpose:**
Generates all possible `(context, target)` pairs for training the CBOW model.

---

## 🧩 **CELL 5 — Convert Lists to NumPy Arrays**

```python
dataX = np.array(dataX)
dataY = np.array(dataY)
```

### 🔹 Line-by-Line Explanation:

1. `dataX = np.array(dataX)`
   → Converts list of lists (contexts) into a 2D NumPy array for easier model input.

2. `dataY = np.array(dataY)`
   → Converts list of target words into a NumPy array.

**Purpose:** NumPy arrays are faster and compatible with Keras models.

---

## 🧩 **CELL 6 — Building the CBOW Neural Network Model**

```python
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Lambda, Dense
import tensorflow.keras.backend as K

model = Sequential()
model.add(Embedding(V+1, 10, input_length=window_size*2))
model.add(Lambda(lambda x: K.mean(x, axis=1), output_shape=(10,)))
model.add(Dense(V, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
```

### 🔹 Line-by-Line Explanation:

1. `from tensorflow.keras.models import Sequential`
   → Imports Keras Sequential model class (used to stack layers linearly).

2. `from tensorflow.keras.layers import Embedding, Lambda, Dense`
   → Imports layers used:

   * **Embedding:** converts word IDs to dense vectors
   * **Lambda:** applies custom operations (like averaging)
   * **Dense:** fully connected layer (output)

3. `import tensorflow.keras.backend as K`
   → Imports Keras backend functions (for custom operations like mean).

4. `model = Sequential()`
   → Initializes a sequential model (layer-by-layer neural network).

5. `model.add(Embedding(V+1, 10, input_length=window_size*2))`
   → Adds an embedding layer:

   * Input dimension: `V+1` (vocab size)
   * Output dimension: 10 (embedding vector size)
   * Input length: `window_size*2` (4 context words)

6. `model.add(Lambda(lambda x: K.mean(x, axis=1), output_shape=(10,)))`
   → Averages all 4 context word embeddings into a single vector (mean pooling).

7. `model.add(Dense(V, activation='softmax'))`
   → Output layer with `V` neurons, one for each word in vocabulary.
   → `softmax` outputs probability for each word being the target.

8. `model.compile(loss='categorical_crossentropy', optimizer='adam')`
   → Compiles model using categorical cross-entropy loss and Adam optimizer.

**Purpose:**
Defines the CBOW architecture:

> Embedding → Mean (context vector) → Dense (Softmax output)

---

## 🧩 **CELL 7 — Converting Target Words to One-Hot Vectors**

```python
y = to_categorical(dataY - 1, V)
```

### 🔹 Line-by-Line Explanation:

1. `dataY - 1`
   → Since tokenizer starts indexing from 1, we subtract 1 to shift to 0-based indexing.

2. `to_categorical(..., V)`
   → Converts each target word ID into a one-hot encoded vector of size `V`.

**Example:**
If vocabulary size = 10 and target word index = 3,
then one-hot vector = `[0, 0, 1, 0, 0, 0, 0, 0, 0, 0]`

**Purpose:**
Transforms numerical targets into categorical format for softmax classification.

---

## 🧩 **CELL 8 — Training the Model**

```python
model.fit(dataX, y, epochs=500, verbose=0)
```

### 🔹 Line-by-Line Explanation:

1. `model.fit(dataX, y, epochs=500, verbose=0)`
   → Trains the CBOW model for 500 epochs (iterations over dataset).
   → `verbose=0` hides training logs for cleaner output.

**Purpose:**
Model learns word embeddings and the relationship between context and target words.

**Result:**
Each word now has a learned 10-dimensional embedding that captures its meaning relative to others.

---

## 🧠 SUMMARY (Final Understanding)

**CBOW Workflow in Your Code:**

1. Import libraries → preprocessing tools
2. Prepare text dataset → split into words
3. Tokenize text → get integer IDs
4. Generate `(context, target)` pairs
5. Build neural network → Embedding → Mean → Softmax
6. Train model → learns word embeddings



'''